In [1]:
import pandas as pd 
import json 
import numpy as np 
import pdal 
import warnings 
warnings.filterwarnings('ignore')
from pyproj import Proj, transform
from shapely.geometry import Polygon, Point, mapping
import geopandas as gpd


In [2]:
def json_load(filepath):
    """
    - function to load json file 
    """
    try:
        with open(filepath) as file:
            json_file = json.load(file)
        return json_file
    except FileNotFoundError:
        print('this is file is not found')

In [3]:
def convert_EPSG(fromT, lon, lat):
    """
    -- function to change to the needed CRS format for better visualization
    parameters:
    
    1. fromT: original EPSG format
    2. lon: longitude
    3. lat: latitude
    
    WGS84 is the most common global CRS in latitude and longitude and it have
    "init=epsg4326" that will be used in this function 
    
    """
    
    P3857 = Proj(init='epsg:3857')
    P4326 = Proj(init='epsg:4326')
    if(fromT == 4326):
        input1 = P4326
        input2 = P3857
        
    else:
        input1 = p3857
        input2 = p4326
        
    x, y = transform(input1,input2,lon,lat)
    return [x, y]

In [4]:
def loop_EPSG_converter(List):
    """
    -- function to change format of a list of cordinates to a list of points 
    """
    converted_list = []
    for i in List:
        converted_list.append(convert_EPSG(4326, i[0], i[1]))
        
    return converted_list 
        

In [5]:
coordinates = [
    [-93.756055, 41.918115],
    [-93.756155, 41.918215],
    [-93.756396, 41.918475],
    [-93.755955, 41.918300],
    [-93.755795, 41.918000],
]
# change coordinates to a list of points
corr_list = loop_EPSG_converter(coordinates)
# polygon = generate_polygon(coordinates, 4326)
corr_list

[[-10436876.301386151, 5148721.349314567],
 [-10436887.43333523, 5148736.309605352],
 [-10436914.26133251, 5148775.206471106],
 [-10436865.169437071, 5148749.025870951],
 [-10436847.358318545, 5148704.145009145]]

In [6]:
def generate_polygon(coor, epsg):
    """
    --Generate a polygon given a co-ordinate and CRS format

    """
    polygon_g = Polygon(coor)
    crs = {'init': 'epsg:'+str(epsg)}
    polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_g])       
    return polygon

In [7]:
polygon = generate_polygon(corr_list,4326)
polygon

,geometry
0,"POLYGON ((-10436876.30139 5148721.34931, -1043..."


In [8]:
def json_modification(json_filepath, url, region,espg_in, espg_out):
    """
    -- function to modify json pipeline file to fetch data 
    """
    d = json_load(json_filepath)
    d['pipeline'][0]['polygon'] = str(polygon.iloc[:,0][0])
    d['pipeline'][0]['filename'] = f"{url}/{region}/ept.json"
    d['pipeline'][2]['in_srs'] = f"EPSG:{espg_in}"
    d['pipeline'][2]['out_srs'] = f"EPSG:{espg_out}"
    return d
    

In [9]:
path = "../pipeline_json.json"
url = "https://s3-us-west-2.amazonaws.com/usgs-lidar-public"
region = "IA_FullState"
in_srs = 3857
out_srs = 4326

In [10]:
# modifiy json file 
r = json_modification(path, url, region, in_srs, out_srs)

In [11]:
r

{'pipeline': [{'bounds': '([-10425171.940, -10423171.940], [5164494.710, 5166494.710])',
   'filename': 'https://s3-us-west-2.amazonaws.com/usgs-lidar-public/IA_FullState/ept.json',
   'type': 'readers.ept',
   'tag': 'readdata',
   'polygon': 'POLYGON ((-10436876.301386151 5148721.349314567, -10436887.43333523 5148736.309605352, -10436914.26133251 5148775.206471106, -10436865.169437071 5148749.025870951, -10436847.358318545 5148704.145009145, -10436876.301386151 5148721.349314567))'},
  {'limits': 'Classification![7:7]',
   'type': 'filters.range',
   'tag': 'nonoise'},
  {'assignment': 'Classification[:]=0',
   'tag': 'wipeclasses',
   'type': 'filters.assign',
   'in_srs': 'EPSG:3857',
   'out_srs': 'EPSG:4326'},
  {'out_srs': 'EPSG:26915',
   'tag': 'reprojectUTM',
   'type': 'filters.reprojection'},
  {'tag': 'groundify', 'type': 'filters.smrf'},
  {'limits': 'Classification[2:2]',
   'type': 'filters.range',
   'tag': 'classify'},
  {'filename': 'iowa.laz',
   'inputs': ['classif

In [12]:
# pipe creation
pipe = pdal.Pipeline(json.dumps(r))

In [13]:
pipe.execute()

RuntimeError: filters.assign: Unexpected argument 'in_srs'.

In [ ]:
# generate the dataframe  from the pipeline using created pipe 

def geo_df(pipe,epsg):
    elevations = []
    geometry_ = []
    counts = pipe.execute()
    arrays = pipe.arrays[0]
    
    for i in arrays:
        lists = list(i)[-3:]
        elevations.append(lists[2])
        geometry_.append(Point(lists[0],lists[1]))
        
    df = gpd.GeoDataFrame(columns=["elevation_m", "Geometry"])
    df['elevation_m'] = elevations
    df['Geometry'] = geometry_
    df = df.set_geometry('Geometry')
    df.set_crs(epsg = epsg, inplace=True)
    
    return df 

In [ ]:
df = geo_df(pipe, 4326)